In [1]:
from __future__ import division

from models import *
from utils.logger import *
from utils.utils import *
from utils.datasets import *
from utils.parse_config import *
#from test import evaluate
#from test import *
from eval import evaluate

from terminaltables import AsciiTable

import os
import sys
import time
import datetime
import argparse
import easydict

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim

In [2]:
opt = easydict.EasyDict({
    "epochs": 100,
    "batch_size": 4,
    "gradient_accumulations": 2,
    "model_def": "config/yolov3-axle.cfg",
    "data_config": "config/axleDetection.data",
    "pretrained_weights": "",
    "n_cpu": 8,
    "img_size": 416,
    "checkpoint_interval": 1,
    "evaluation_interval": 1,
    "compute_map": False,
    "multiscale_training": True
})

In [ ]:
def train():
    #print(opt)
    
    # Iinitialize Tensorboard
    logger = Logger("logs")
    
    # Select Env
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    os.makedirs("output", exist_ok=True)
    os.makedirs("checkpoints", exist_ok=True)

    # Get data configuration
    data_config = parse_data_config(opt.data_config)
    train_path = data_config["train"]
    valid_path = data_config["valid"]
    class_names = load_classes(data_config["names"])
    
    # Initiate model
    model = Darknet(opt.model_def).to(device)
    model.apply(weights_init_normal)
    
    # If specified we start from checkpoint
    if opt.pretrained_weights:
        if opt.pretrained_weights.endswith(".pth"):
            model.load_state_dict(torch.load(opt.pretrained_weights))
        else:
            model.load_darknet_weights(opt.pretrained_weights)
            
            
    # Get dataloader
    dataset = ListDataset(train_path, augment=False, multiscale=opt.multiscale_training)
    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=opt.batch_size,
        shuffle=True,
        num_workers=opt.n_cpu,
        pin_memory=True,
        collate_fn=dataset.collate_fn,
    )
    
    optimizer = torch.optim.Adam(model.parameters())

    metrics = [
        "grid_size",
        "loss",
        "x",
        "y",
        "w",
        "h",
        "conf",
        "cls",
        "cls_acc",
        "recall50",
        "recall75",
        "precision",
        "conf_obj",
        "conf_noobj",
    ]
    
    for epoch in range(opt.epochs):
        model.train()
        start_time = time.time()
        for batch_i, (_, imgs, targets) in enumerate(dataloader):
            batches_done = len(dataloader) * epoch + batch_i

            imgs = Variable(imgs.to(device))
            targets = Variable(targets.to(device), requires_grad=False)

            loss, outputs = model(imgs, targets)
            loss.backward()

            if batches_done % opt.gradient_accumulations:
                # Accumulates gradient before each step
                optimizer.step()
                optimizer.zero_grad()

            # ----------------
            #   Log progress
            # ----------------

            log_str = "\n---- [Epoch %d/%d, Batch %d/%d] ----\n" % (epoch, opt.epochs, batch_i, len(dataloader))

            metric_table = [["Metrics", *[f"YOLO Layer {i}" for i in range(len(model.yolo_layers))]]]

            # Log metrics at each YOLO layer
            for i, metric in enumerate(metrics):
                formats = {m: "%.6f" for m in metrics}
                formats["grid_size"] = "%2d"
                formats["cls_acc"] = "%.2f%%"
                row_metrics = [formats[metric] % yolo.metrics.get(metric, 0) for yolo in model.yolo_layers]
                metric_table += [[metric, *row_metrics]]

                # Tensorboard logging
                tensorboard_log = []
                for j, yolo in enumerate(model.yolo_layers):
                    for name, metric in yolo.metrics.items():
                        if name != "grid_size":
                            tensorboard_log += [(f"{name}_{j+1}", metric)]
                tensorboard_log += [("loss", loss.item())]
                logger.list_of_scalars_summary(tensorboard_log, batches_done)

            log_str += AsciiTable(metric_table).table
            log_str += f"\nTotal loss {loss.item()}"

            # Determine approximate time left for epoch
            epoch_batches_left = len(dataloader) - (batch_i + 1)
            time_left = datetime.timedelta(seconds=epoch_batches_left * (time.time() - start_time) / (batch_i + 1))
            log_str += f"\n---- ETA {time_left}"

            print(log_str)

            model.seen += imgs.size(0)

        if epoch % opt.evaluation_interval == 0:
            print("\n---- Evaluating Model ----")
            # Evaluate the model on the validation set
            precision, recall, AP, f1, ap_class = evaluate(
                model,
                path=valid_path,
                iou_thres=0.5,
                conf_thres=0.5,
                nms_thres=0.5,
                img_size=opt.img_size,
                batch_size=8,
            )
            evaluation_metrics = [
                ("val_precision", precision.mean()),
                ("val_recall", recall.mean()),
                ("val_mAP", AP.mean()),
                ("val_f1", f1.mean()),
            ]
            logger.list_of_scalars_summary(evaluation_metrics, epoch)

            # Print class APs and mAP
            ap_table = [["Index", "Class name", "AP"]]
            for i, c in enumerate(ap_class):
                ap_table += [[c, class_names[c], "%.5f" % AP[i]]]
            print(AsciiTable(ap_table).table)
            print(f"---- mAP {AP.mean()}")

        if epoch % opt.checkpoint_interval == 0:
            torch.save(model.state_dict(), f"checkpoints/yolov3_ckpt_%d.pth" % epoch)
    
train()

/home/seungtaek/PyTorch-YOLOv3/models.py:217: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)
  loss_x = self.mse_loss(x[obj_mask], tx[obj_mask])
/home/seungtaek/PyTorch-YOLOv3/models.py:218: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)
  loss_y = self.mse_loss(y[obj_mask], ty[obj_mask])
/home/seungtaek/PyTorch-YOLOv3/models.py:219: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)
  loss_w = self.mse_loss(w[obj_mask], tw[obj_mask])
/home/seungtaek/PyTorch-YOLOv3/models.py:220: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered intern


---- [Epoch 0/100, Batch 0/126] ----
+------------+--------------+--------------+--------------+
| Metrics    | YOLO Layer 0 | YOLO Layer 1 | YOLO Layer 2 |
+------------+--------------+--------------+--------------+
| grid_size  | 15           | 30           | 60           |
| loss       | 77.239609    | 78.023705    | 76.252007    |
| x          | 0.114614     | 0.109907     | 0.121856     |
| y          | 0.079215     | 0.077566     | 0.088672     |
| w          | 0.587513     | 0.490010     | 0.512348     |
| h          | 0.780567     | 0.479901     | 1.331826     |
| conf       | 74.893387    | 76.149750    | 73.492538    |
| cls        | 0.784319     | 0.716569     | 0.704766     |
| cls_acc    | 0.00%        | 7.69%        | 7.69%        |
| recall50   | 0.000000     | 0.076923     | 0.000000     |
| recall75   | 0.000000     | 0.000000     | 0.000000     |
| precision  | 0.000000     | 0.000132     | 0.000000     |
| conf_obj   | 0.468709     | 0.537191     | 0.421913     |
| 

Detecting objects:   0%|          | 0/63 [00:00<?, ?it/s]


---- [Epoch 0/100, Batch 125/126] ----
+------------+--------------+--------------+--------------+
| Metrics    | YOLO Layer 0 | YOLO Layer 1 | YOLO Layer 2 |
+------------+--------------+--------------+--------------+
| grid_size  | 14           | 28           | 56           |
| loss       | 2.660771     | 2.851617     | 3.008796     |
| x          | 0.027660     | 0.120821     | 0.079085     |
| y          | 0.066002     | 0.117111     | 0.054914     |
| w          | 0.097699     | 0.022201     | 0.048911     |
| h          | 0.154569     | 0.060213     | 0.070811     |
| conf       | 2.196914     | 2.431044     | 2.652997     |
| cls        | 0.117927     | 0.100227     | 0.102079     |
| cls_acc    | 66.67%       | 66.67%       | 66.67%       |
| recall50   | 0.000000     | 0.222222     | 0.555556     |
| recall75   | 0.000000     | 0.111111     | 0.444444     |
| precision  | 0.000000     | 0.142857     | 0.071429     |
| conf_obj   | 0.275981     | 0.380857     | 0.650988     |


Computing AP: 100%|██████████| 7/7 [00:00<00:00, 703.47it/s]


...

Computing AP: 100%|██████████| 7/7 [00:00<00:00, 1005.83it/s]


+-------+---------------+---------+
| Index | Class name    | AP      |
+-------+---------------+---------+
| 0     | NormalCar     | 1.04919 |
| 1     | Bus           | 0.18265 |
| 2     | SmallTruckA   | 0.02159 |
| 3     | SmallTruckB   | 0.06946 |
| 4     | MidSizeTruckA | 0.13137 |
| 5     | MidSizeTruckB | 0.00000 |
| 6     | MidSizeTruckC | 0.03743 |
+-------+---------------+---------+
---- mAP 0.21309842136533788

---- [Epoch 5/100, Batch 0/126] ----
+------------+--------------+--------------+--------------+
| Metrics    | YOLO Layer 0 | YOLO Layer 1 | YOLO Layer 2 |
+------------+--------------+--------------+--------------+
| grid_size  | 16           | 32           | 64           |
| loss       | 1.189510     | 1.216693     | 1.139581     |
| x          | 0.036138     | 0.050638     | 0.054815     |
| y          | 0.060617     | 0.048065     | 0.032215     |
| w          | 0.020955     | 0.021112     | 0.006363     |
| h          | 0.016509     | 0.017951     | 0.023834    

Detecting objects:   0%|          | 0/63 [00:00<?, ?it/s]


---- [Epoch 5/100, Batch 125/126] ----
+------------+--------------+--------------+--------------+
| Metrics    | YOLO Layer 0 | YOLO Layer 1 | YOLO Layer 2 |
+------------+--------------+--------------+--------------+
| grid_size  | 16           | 32           | 64           |
| loss       | 0.834233     | 1.094919     | 0.731676     |
| x          | 0.008830     | 0.037544     | 0.035439     |
| y          | 0.017110     | 0.070696     | 0.054862     |
| w          | 0.056934     | 0.010331     | 0.034304     |
| h          | 0.006760     | 0.013665     | 0.042840     |
| conf       | 0.659776     | 0.868341     | 0.487961     |
| cls        | 0.084822     | 0.094343     | 0.076269     |
| cls_acc    | 66.67%       | 66.67%       | 66.67%       |
| recall50   | 0.666667     | 0.333333     | 0.666667     |
| recall75   | 0.666667     | 0.166667     | 0.333333     |
| precision  | 0.307692     | 0.086957     | 0.072727     |
| conf_obj   | 0.845167     | 0.691623     | 0.921481     |


Detecting objects:  17%|█▋        | 11/63 [00:22<01:47,  2.07s/it]